```
@title : Tracker X data analysis
@date  : 20250212 ALUR
@author: Aleksandras Urbonas
```


---
#
# Step 1. Load the data
Load the files into `Pandas DataFrames`.

Data analyzed: Activity Sessions, Daily Health Metrics. Excluded: Health Metrics Samples

Data descriptions are limited, see <data\schema.md>.
#

In [ ]:


import pandas as pd  # load and process data
import numpy as np  # process numeric values



---
##
## Daily
##

In [ ]:


# Load the data
daily__raw = pd.read_csv(
    '../data/00_original/health_metrics_daily_2025-01-28.csv'
# , index_col='PK_HEALTH_METRICS_HEART_ID'
)

# preview
daily__raw.head(2)



In [ ]:


# working dataset
daily__dev = daily__raw.copy()



In [ ]:


# columns check
daily__dev.info()



In [ ]:


# only keep valid rows
daily__dev = daily__dev[daily__dev['ROW_IS_VALID'] == 1]

del daily__dev['ROW_IS_VALID']



In [ ]:


# drop timezone: already included
del daily__dev['TIMEZONE_OFFSET']



In [ ]:


# drop system timestamps
del daily__dev['CREATED_DATE']
del daily__dev['LAST_MODIFIED_DATE']



In [ ]:


# A-1. invalid date in on row:
drop_daily_date_idx = daily__dev[daily__dev['DATE'] == '0001-01-01'].index
# FK_USER_ID=21775, PK_HEALTH_METRICS_HEART_ID=790

# A-2. drop invalid row
daily__dev = daily__dev.drop(drop_daily_date_idx)



In [ ]:


# B. convert datetime
daily__dev['DATE'] = pd.to_datetime(daily__dev['DATE']) #, format='mixed')
daily__dev['DATE'] = daily__dev['DATE'].dt.date #, format='mixed')

# Error:
# OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 0001-01-01, at position 5. You might want to try:
#     - passing `format` if your strings have a consistent format;
#     - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
#     - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

# rename date column:
daily__dev.rename({'DATE':'DAILY_DATE'}, axis='columns', inplace=True)



In [ ]:


# Check for missing data, %
print(daily__dev.isnull().sum() * 100 / daily__dev.shape[0])



In [ ]:


# A-1. invalid date in on row:
drop_daily_avg_hr_idx = daily__dev[daily__dev['AVG_HR'].isnull()].index
# FK_USER_ID=21775, PK_HEALTH_METRICS_HEART_ID=790

# A-2. drop invalid row
daily__dev = daily__dev.drop(drop_daily_avg_hr_idx)



In [ ]:


del daily__dev['HEART_RATE_VARIABILITY_DAY_HRV']  # all null
del daily__dev['HEART_RATE_VARIABILITY_SLEEP_HRV']  # all null



In [ ]:


# review missing data
# daily__dev[daily__dev['RESTING_HR'].isnull()].head(1)
# 15-20% are null negative timezeone?

# removing: out of scope. focus on AVG_HR.
del daily__dev['RESTING_HR']
del daily__dev['MIN_HR']
del daily__dev['MAX_HR']



###
### Select users for next analysis
###

In [ ]:


# Select first user in data
# users_sample_list = 14064

# Randomly sample N users
# users_sample_list = daily__dev['FK_USER_ID'].sample(n=1000, random_state=88888888).to_list()
# # use list from Sessions
# print(f"* Users sampled: {users_sample_list[:3]}")

# select sample of users
# daily__user = daily__dev[daily__dev['FK_USER_ID'].isin(users_sample_list)]

# or go full
daily__user = daily__dev
daily__user.reset_index(drop=True, inplace=True)
print(f"* Rows selected: {daily__user.shape[0]}")



In [ ]:


#summary
daily__user.describe()



In [ ]:


# write to file for analysis
daily__user.to_csv('../data/01_review/daily__user.csv', index=False)



In [ ]:


# Preview
daily__user.head(2)

# 143 records
# FK: PK_HEALTH_METRICS_HEART_ID



#### **Insights for `Daily` Dataset**

Dataset `Daily` is prepared:
- Variables included are the User, the date, the Average HR, and the Source.
- Several analysis can be performed from such data.



---
##
## Sessions
##

In [ ]:


# Load the data
sessions__raw = pd.read_csv(
    '../data/00_original/activity_sessions_2025-01-28.csv'
# , index_col='PK_ACTIVITY_SESSION_ID'
, low_memory=False  # DtypeWarning: Columns (13,14) have mixed types
)

# preview
sessions__raw.head(2)



In [ ]:


# working copy
sessions__dev = sessions__raw.copy()



In [ ]:


# review dataset
sessions__dev.info()



In [ ]:


# Check for missing data
print(sessions__dev.isnull().sum())
# INTENSITY                  684883
# RATING                    1491032
# RATING_ISSUE              1928737
# RATING_ISSUE_MESSAGE      1930085



In [ ]:


# include only valid rows
sessions__dev = sessions__dev[sessions__dev['ROW_IS_VALID'] == 1]
# delete redundant columns:
del sessions__dev['ROW_IS_VALID']



In [ ]:


# include: only finished sessions
sessions__dev = sessions__dev[sessions__dev['IS_FINISHED'] == 1]
# delete redundant columns
del sessions__dev['IS_FINISHED']



In [ ]:


# remove primary key, unused:
del sessions__dev['PK_ACTIVITY_SESSION_ID']



In [ ]:


# delete redundant system columns
del sessions__dev['CREATED_DATE']  # end of session: out of scope
del sessions__dev['LAST_MODIFIED_DATE']  # use for modifications: out of scope



In [ ]:


# Ratings: few records have such info: DECISION to remove from current analysis
del sessions__dev['RATING']
del sessions__dev['RATING_ISSUE']
del sessions__dev['RATING_ISSUE_MESSAGE']



In [ ]:


# FK_SYSTEM_PROGRAM_ID == FK_REFERENCE_ID?
print(sessions__dev['FK_SYSTEM_PROGRAM_ID'][sessions__dev['FK_SYSTEM_PROGRAM_ID'] != sessions__dev['FK_REFERENCE_ID']].count())

# FK_SYSTEM_PROGRAM_ID=0, when INTENSITY variable is populated.
# So, FK_REFERENCE_ID is more specific.
# DECISION: remove less specific variable
del sessions__dev['FK_SYSTEM_PROGRAM_ID']



In [ ]:


# convert to datetime 
sessions__dev['START_DATETIME'] = pd.to_datetime(sessions__dev['START_DATE'])
sessions__dev['END_DATETIME'] = pd.to_datetime(sessions__dev['END_DATE'])

# del sessions__dev['START_DATE']
del sessions__dev['END_DATE']

# extract dates from datetime
sessions__dev['START_DATE'] = sessions__dev['START_DATETIME'].dt.date
# sessions__dev['END_DATE'] = sessions__dev['END_DATETIME'].dt.date



In [ ]:


# Calc: Add time-based features for analysis
sessions__dev['SESSION_DURATION'] = ( (sessions__dev['END_DATETIME'] - sessions__dev['START_DATETIME']).dt.total_seconds() / 60).round(2)
# preview
sessions__dev['SESSION_DURATION'].head(2)

# drop used columns
del sessions__dev['START_DATETIME']
del sessions__dev['END_DATETIME']



###
### Select users
###

In [ ]:

# sample one: first
# users_sample_list = 14064
# sample from Daily: users
# users_sample_list = [36899, 57417, 13844]
# sample random
# users_sample_list = sessions__dev['FK_USER_ID'].sample(n=3, random_state=88888888).to_list()
# print(f"* Selected users: {users_sample_list[:3]} ...")

# select data for sampled users
# sessions__user = sessions__dev[sessions__dev['FK_USER_ID'].isin(users_sample_list)]

# or go full
sessions__user = sessions__dev
print(f"* Selected rows: {sessions__user.shape[0]}")



In [ ]:


# check some basic statistics
sessions__user.describe()



In [ ]:


# write to file for analysis
sessions__user.to_csv('../data/01_review/sessions__user.csv')





### Insights

Sessions of Selected Users were exported to a file and reviewed manually.
- Two users have data.
- One used has no data.



In [ ]:


sessions__user.head(2)



##
## Step 3: Check the column names to ensure foreign key matching
##

In [ ]:


# Clean and validate the column names
sessions__user.columns = sessions__user.columns.str.strip()  # Remove any extra spaces
daily__user.columns = daily__user.columns.str.strip()

print("Sessions Columns:", sessions__user.columns)
print("Daily Columns:", daily__user.columns)



#
# Step 4. Merge Sessions with Daily measurements
#

In [ ]:


merged_data = pd.merge(
    daily__user
    , sessions__user
    , left_on=['FK_USER_ID', 'DAILY_DATE']
    , right_on=['FK_USER_ID', 'START_DATE']
    , how='left'
)
# drop START_DATE
del merged_data['START_DATE']
print(f"Data merged: {merged_data.shape[0]} rows and {merged_data.shape[1]} columns.")



In [ ]:


# data structure
merged_data.info()



#
# Step 5: Data exploration to understand the relationships and values
#


In [ ]:


merged_data.head(2)





### Manual Review

Analysis of merged Daily and Sessions data for selected users:

- 13844: stimulated 0 times
- 36899: stimulated 0 times
- 57417: stimulated multiple times



#
# Step 6: Filter samples based on the session start and end dates (skip)
#

In [ ]:


merged_data.shape



#
# Step 7: Data Cleaning and Transformation
#

In [ ]:


# Handle missing values and data anomalies
X_to_clean = ['AVG_HR'] #, 'DATE']
merged_data.dropna(subset=X_to_clean, inplace=True)
merged_data.shape




##
## Step 8: Normalize the AVG_HR
##

In [ ]:


merged_data['AVG_HR'] = merged_data['AVG_HR'].round(1)  # Round to 1 decimal place
# Alternatively, if you want integers:
# merged_data['AVG_HEART_RATE'] = merged_data['AVG_HEART_RATE'].astype(int)
merged_data.head(2)



#
# Step 9: Feature Engineering
#

In [ ]:


# Calc: `IS_INTENSITY`: 1=Yes, 0=No
x_in = 'INTENSITY'
x_out = 'IS_INTENSITY'
if x_in in merged_data.columns:
    merged_data.loc[merged_data[x_in].isna(), x_out] = 0
    merged_data.loc[~merged_data[x_in].isna(), x_out] = 1
    merged_data[x_out] = merged_data[x_out].astype(int)
    merged_data.drop(columns=x_in, inplace=True)

merged_data.head(2)



In [ ]:


# export merged data
merged_data.to_csv('../data/01_review/merged_data.csv')



##
## Step 10: Identify the impact of X stimulation on heart rate
##

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# set chart size
plt.rcParams["figure.figsize"] = 5, 3
sns.set_theme(rc={'figure.figsize':(5, 3)})



In [ ]:


sns.boxplot(x='IS_INTENSITY', y='AVG_HR', data=merged_data)
# plt.title('AVG_HR by Stimulation (1=Yes)')
plt.show()



In [ ]:


# Heart by user:
merged_data.groupby(['FK_USER_ID', 'IS_INTENSITY'])['AVG_HR'].mean()



#
# Advanced statistics: compare Stimulated and non-Stimulated
#

In [ ]:
from scipy import stats

x = 'IS_INTENSITY'
y = 'AVG_HR'

# Separate performance scores by gender
stimulated_Y_performance = merged_data[merged_data[x] == 1][y]
stimulated_N_performance = merged_data[merged_data[x] == 0][y]

# Perform a t-test for performance scores by gender 
t_stat_performance, p_val_performance = stats.ttest_ind(stimulated_Y_performance, stimulated_N_performance)
print(f"\nT-test for y={y} by x={x}:")
print(f"T-statistic: {np.round(t_stat_performance, 3)}, p-value: {np.round(p_val_performance, 3)}")

# Check if the p-value is less than 0.05 for significance
if p_val_performance < 0.05: print(f"There is a significant difference.")
else: print(f"No significant difference.")



In [ ]:


# For simplicity, we assume sessions with 'Intensity' involve stimulation: already filtered
stimulated_data = merged_data[['IS_INTENSITY', 'AVG_HR', 'FK_USER_ID', 'SESSION_DURATION']]
stimulated_data.head(2)



In [ ]:


# Primary Question: Does X stimulation affect heart rate?
stimulated_data['HR_impact'] = stimulated_data.groupby(['FK_USER_ID', 'IS_INTENSITY'])['AVG_HR'].diff()  # Difference in HR
stimulated_data['HR_impact']



##
## Step 11: Aggregate and analyze data
##

In [ ]:


# Aggregate average HR change for each user and session
HR_analysis = stimulated_data.groupby('FK_USER_ID').agg(
    avg_impact_hr=('HR_impact', 'mean'),
    max_impact_hr=('HR_impact', 'max'),
    session_count=('IS_INTENSITY', 'count'),
    avg_session=('SESSION_DURATION', 'mean'),
).reset_index()

print(f"* Users in HR analysis: {HR_analysis.shape[0]}")



In [ ]:


# Additional Insights: Identify any patterns based on time or other factors
HR_analysis['impact_sign'] = np.sign(HR_analysis['avg_impact_hr'])  # Positive or Negative Impact

print("HR Analysis - Impact of X Stimulation:")
print(HR_analysis.head())



##
## Step 12: Save the results or create a detailed report
##

In [ ]:


x = "session_count"
SESSIONS_COUNT_LIMIT = 10 

# before
count_before = HR_analysis.shape[0]

# some users have few sessions
HR_analysis = HR_analysis[HR_analysis[x] >= SESSIONS_COUNT_LIMIT]

# after
count_after = HR_analysis.shape[0]

print(f"* Filter by x={x}: was {count_before}, now {count_after}")



In [ ]:


HR_analysis.to_csv('../data/01_review/HR_analysis_results.csv', index=False)



##
## Step 13: Report
##

In [ ]:


report = """
@title : Impact of Stimulation on Heart Rate
@date  : {report_date}
@author: Aleksandras Urbonas

1. Primary Question: Does X stimulation affect heart rate metrics?
- Based on the analysis, we found that users (N={N_subjects}) who participated in the 'Program' - indicating stimulation - (at least S={SESSIONS_COUNT_LIMIT} sessions) - showed an average HR change of {avg_impact:.2f} bpm. 
- The impact varied across sessions, with the maximum observed change being {max_impact:.2f} bpm.

2. Secondary Insights:
- Additional factors influencing HR change include session duration and time of day (further analysis needed for seasonality patterns).
- A significant amount of time was dedicated to data exploration and schema understanding. A more detailed schema and business process description can be of help.
- Samples data contains ~70 mln records, which slows the analysis and it is recommended to process such data in SQL, for example, performing aggregations by user over different periods of time.
- Data validation was completed: some records were excluded from analysis.
- Data was analysed using Python, allowing the analysis to be repeated.
""".format(
    N_subjects = HR_analysis['FK_USER_ID'].count()
    , avg_impact=HR_analysis['avg_impact_hr'].mean()
    , max_impact=HR_analysis['max_impact_hr'].max()
    , report_date=pd.to_datetime('today').strftime('%Y-%m-%d')
    , SESSIONS_COUNT_LIMIT=SESSIONS_COUNT_LIMIT
    , avg_session=HR_analysis['avg_session'].max()
)


print(report)



In [ ]:


# save report as file:
with open('../report.md', 'w') as _file:
    _file.write(report)



### Insights

Several users (for example, 1532) has no average session duration: such records must be excluded.



In [ ]:


# analysis data for a single user without [avg_session] 
HR_analysis[HR_analysis['FK_USER_ID']==1532]



In [ ]:


# find all users without [avg_session]
HR_analysis[HR_analysis['avg_session'].isna()]
# 326 rows

